## 1.Cài đặt thư viện cần thiết

In [2]:
import os

# ======================================================
# LOAD CHECKPOINT TỪ ĐƯỜNG DẪN CỤ THỂ
# ======================================================
checkpoint_path = r"C:\NCKH\XAI\LogCheckpointDDPG\ddpg_paper_final_3938000_steps.zip"

if os.path.exists(checkpoint_path):
    print(f"✅ Tìm thấy checkpoint: {checkpoint_path}")
    print(f"📊 Timesteps đã train: 3,938,000")
    INPUT_CHECKPOINT_PATH = checkpoint_path
    print(f"✔️ Đã gán INPUT_CHECKPOINT_PATH = {INPUT_CHECKPOINT_PATH}")
else:
    print(f"❌ Không tìm thấy file tại: {checkpoint_path}")
    print("⚠️ Sẽ tạo model mới nếu không có checkpoint khác")

✅ Tìm thấy checkpoint: C:\NCKH\XAI\LogCheckpointDDPG\ddpg_paper_final_3938000_steps.zip
📊 Timesteps đã train: 3,938,000
✔️ Đã gán INPUT_CHECKPOINT_PATH = C:\NCKH\XAI\LogCheckpointDDPG\ddpg_paper_final_3938000_steps.zip


In [1]:
# Cài đặt Stable Baselines3 và các phụ thuộc
!pip install stable-baselines3[extra] gymnasium shimmy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 8.7 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of shimmy[atari] to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 952.1/952.1 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.0/188.0 kB 13.8 MB/s eta 0:00:00
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 0.29.0
    Uninstalling gymnasium-0.29.0:
      Successfully uninstalled gymnasium-0.29.0
  Attempting uninstall: stable-baselines3
    Found existing installation: stable-baselines3 2.1.0
    Uninstalling stable-baselines3-2.1.0:
      Successfully uninstalled stable-baselines3-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installe

## 2. Cấu hình & Chuyển đổi dữ liệu (CSV -> TFRecords)

In [2]:
import os
import shutil
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime
import random

# --- CẤU HÌNH ---
class Config:
    OUTPUT_DIR = '/kaggle/working/data220'
    NUMBER_OF_PRODUCTS = 220
    START_TIME_PERIOD = 0
    MIDDLE_TIME_PERIOD = 900 
    END_TIME_PERIOD = 1396   

cfg = Config()

def restore_or_generate_data():
    # 1. Thử tìm dữ liệu trong Input để copy (nhanh)
    input_root = '/kaggle/input'
    found_path = None
    for dirname, _, filenames in os.walk(input_root):
        if dirname.endswith('data220') and 'train.tfrecords' in filenames:
            found_path = dirname
            break
            
    if found_path:
        print(f"✅ Tìm thấy dữ liệu cũ tại: {found_path}")
        if os.path.exists(cfg.OUTPUT_DIR): shutil.rmtree(cfg.OUTPUT_DIR)
        shutil.copytree(found_path, cfg.OUTPUT_DIR)
        print("✅ Đã copy dữ liệu sang Working!")
        return True
    
    # 2. Nếu không thấy, tạo mới (lâu hơn xíu)
    print("⚠️ Không thấy dữ liệu cũ. Đang tạo mới...")
    if not os.path.exists(cfg.OUTPUT_DIR): os.makedirs(cfg.OUTPUT_DIR)
    
    # (Code tạo dữ liệu gốc - bản rút gọn an toàn)
    def _float_feature(value): return tf.train.Feature(float_list=tf.train.FloatList(value=value))
    def create_tf_example(data_dict): return tf.train.Example(features=tf.train.Features(feature=data_dict))
    
    # Stock
    stock = np.random.uniform(0, 1, cfg.NUMBER_OF_PRODUCTS)
    with tf.io.TFRecordWriter(f'{cfg.OUTPUT_DIR}/stock.tfrecords') as w:
        w.write(create_tf_example({'stock': _float_feature(stock)}).SerializeToString())

    # Orders & Sales (Fix Overflow)
    orders_df = pd.read_csv('/kaggle/input/instacart-market-basket-analysis/orders.csv')
    orders_df = orders_df[orders_df['user_id'] <= 5000].copy()
    orders_df['days_since_prior_order'] = orders_df['days_since_prior_order'].fillna(0)
    orders_df['elapsed_days'] = orders_df.groupby('user_id')['days_since_prior_order'].cumsum()
    orders_df['time_period'] = ((orders_df['elapsed_days'] * 24 + orders_df['order_hour_of_day']) / 6).astype(int)
    orders_df = orders_df[(orders_df['time_period'] >= cfg.START_TIME_PERIOD) & 
                          (orders_df['time_period'] <= cfg.END_TIME_PERIOD)]

    # Products
    products = pd.read_csv('/kaggle/input/instacart-market-basket-analysis/products.csv')
    op_train = pd.read_csv('/kaggle/input/instacart-market-basket-analysis/order_products__train.csv')
    merged = pd.merge(orders_df, op_train, on='order_id')
    top_ids = merged['product_id'].value_counts().head(cfg.NUMBER_OF_PRODUCTS).index.values

    # Capacity
    subset = merged[merged['product_id'].isin(top_ids)]
    counts = subset['product_id'].value_counts().reindex(top_ids, fill_value=1)
    capacities = (counts.values / 10.0).clip(min=5.0)
    with tf.io.TFRecordWriter(f'{cfg.OUTPUT_DIR}/capacity.tfrecords') as w:
        w.write(create_tf_example({'capacity': _float_feature(capacities)}).SerializeToString())

    # Sales Matrix
    sales_matrix = pd.crosstab(subset['time_period'], subset['product_id'])
    sales_matrix = sales_matrix.reindex(index=range(cfg.START_TIME_PERIOD, cfg.END_TIME_PERIOD+1), 
                                        columns=top_ids, fill_value=0).values.astype(float)
    
    with tf.io.TFRecordWriter(f'{cfg.OUTPUT_DIR}/train.tfrecords') as w:
        for row in sales_matrix[cfg.START_TIME_PERIOD : cfg.MIDDLE_TIME_PERIOD]:
            w.write(create_tf_example({'sales': _float_feature(row)}).SerializeToString())
    with tf.io.TFRecordWriter(f'{cfg.OUTPUT_DIR}/test.tfrecords') as w:
        for row in sales_matrix[cfg.MIDDLE_TIME_PERIOD : cfg.END_TIME_PERIOD]:
            w.write(create_tf_example({'sales': _float_feature(row)}).SerializeToString())
            
    print("✅ Tạo dữ liệu hoàn tất!")
    return True

restore_or_generate_data()

2025-12-21 04:29:07.722016: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766291347.932536      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766291347.994262      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766291348.490882      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766291348.490923      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766291348.490926      55 computation_placer.cc:177] computation placer alr

✅ Tìm thấy dữ liệu cũ tại: /kaggle/input/ddgp-s-d-ngsgp/data220
✅ Đã copy dữ liệu sang Working!


True

## 3.Load Dữ liệu vào RAM & Định nghĩa Môi trường

In [3]:
import gymnasium as gym
from gymnasium import spaces

def load_tfrecord_to_numpy(filename, feature_name):
    filepath = os.path.join(cfg.OUTPUT_DIR, filename)
    raw_dataset = tf.data.TFRecordDataset(filepath)
    feature_description = {feature_name: tf.io.VarLenFeature(tf.float32)}
    def _parse(proto): return tf.io.parse_single_example(proto, feature_description)
    parsed = raw_dataset.map(_parse)
    data = []
    for r in parsed: data.append(tf.sparse.to_dense(r[feature_name]).numpy())
    return np.array(data)

print("Loading data into RAM...")
train_sales = load_tfrecord_to_numpy('train.tfrecords', 'sales')
capacities = load_tfrecord_to_numpy('capacity.tfrecords', 'capacity')[0]
capacities[capacities == 0] = 1.0 

# --- ENV DEFINITION ---
class InstacartEnv(gym.Env):
    def __init__(self, sales_data, capacities, num_products):
        super(InstacartEnv, self).__init__()
        self.sales_data = sales_data
        self.capacities = capacities
        self.p = num_products
        self.max_steps = len(sales_data) - 5
        self.current_step = 0
        self.v = np.random.uniform(0.1, 1.0, self.p)
        self.c = np.random.uniform(0.1, 1.0, self.p)
        self.shelf_life = np.random.uniform(0.8, 0.99, self.p) 
        self.v_max = np.sum(self.v) * 0.2 
        self.c_max = np.sum(self.c) * 0.2
        self.action_space = spaces.Box(low=0, high=1, shape=(self.p,), dtype=np.float32)
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(self.p * 8,), dtype=np.float32)
        self.state = np.random.rand(self.p)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = 4 
        self.state = np.random.rand(self.p)
        return self._get_obs(), {}

    def _get_obs(self):
        history = self.sales_data[self.current_step-4 : self.current_step] / self.capacities
        forecast = np.mean(history, axis=0)
        sigma = np.std(history, axis=0)
        obs = np.column_stack((
            self.state, forecast, sigma, self.v, self.c, self.shelf_life,
            np.full(self.p, np.sum(forecast * self.v)), np.full(self.p, np.sum(forecast * self.c))
        )).flatten().astype(np.float32)
        return obs

    def step(self, action):
        vol_req = np.dot(self.v, action)
        weight_req = np.dot(self.c, action)
        rho = max(vol_req / self.v_max, weight_req / self.c_max, 1.0)
        u_actual = action / rho
        inv_after = np.clip(self.state + u_actual, 0, 1)
        sales_norm = self.sales_data[self.current_step] / self.capacities
        wastage = inv_after * (1 - self.shelf_life)
        self.state = np.clip(inv_after - sales_norm - wastage, 0, 1)
        
        is_empty = (self.state <= 0).astype(float)
        fairness = np.percentile(self.state, 95) - np.percentile(self.state, 5)
        capacity_pen = max(rho - 1, 0)
        reward = np.mean(1 - is_empty - wastage) - 0.1 * fairness - 0.5 * capacity_pen
        
        self.current_step += 1
        terminated = self.current_step >= self.max_steps
        return self._get_obs(), reward, terminated, False, {}

Loading data into RAM...


I0000 00:00:1766291408.613671      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1766291408.617624      55 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


## 4. Cấu hình Callback Cuốn chiếu (Giải quyết 19GB Limit)

In [4]:
from stable_baselines3.common.callbacks import BaseCallback
import glob

class RollingCheckpointCallback(BaseCallback):
    def __init__(self, save_freq, save_path, name_prefix='rl_model', max_keep=3, verbose=1):
        super(RollingCheckpointCallback, self).__init__(verbose)
        self.save_freq = save_freq
        self.save_path = save_path
        self.name_prefix = name_prefix
        self.max_keep = max_keep
        os.makedirs(save_path, exist_ok=True)

    def _on_step(self) -> bool:
        if self.n_calls % self.save_freq == 0:
            path = os.path.join(self.save_path, f"{self.name_prefix}_{self.num_timesteps}_steps")
            self.model.save(path)
            self.model.save_replay_buffer(path + "_replay_buffer")
            if self.verbose > 0: print(f"💾 Saved: {path}")
            self._cleanup()
        return True

    def _cleanup(self):
        try:
            files = glob.glob(os.path.join(self.save_path, f"{self.name_prefix}_*.zip"))
            files.sort(key=os.path.getmtime)
            if len(files) > self.max_keep:
                for f in files[:-self.max_keep]:
                    os.remove(f)
                    buf = f.replace(".zip", "_replay_buffer.pkl")
                    if os.path.exists(buf): os.remove(buf)
        except Exception as e: print(f"⚠️ Cleanup error: {e}")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## 5.Huấn luyện DDPG (Có Auto-Resume & Logging)

In [ ]:
import torch as th
import torch.nn as nn
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.monitor import Monitor # <-- Import này quan trọng

# --- CẤU HÌNH ---
LOG_DIR = "/kaggle/working/logs"
CHECKPOINT_DIR = "/kaggle/working/checkpoints"
MODEL_PREFIX = "ddpg_paper_final"

# Nếu bạn muốn chỉ định file cụ thể để resume, điền vào đây. 
# Nếu để None, code sẽ tự tìm trong Input.
INPUT_CHECKPOINT_PATH = None 

# Mục tiêu 7500 Episodes (~6.7 triệu steps)
TOTAL_TIMESTEPS_TARGET = 6700000 
# Giới hạn chạy cho phiên này (ví dụ chạy thêm 2 triệu bước nữa rồi tắt)
STEPS_TO_RUN_THIS_SESSION = 3000000 

# Learning Rate (Dùng mức nhỏ để ổn định vì đã resume)
CURRENT_LR = 0.0001 

os.makedirs(LOG_DIR, exist_ok=True)
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# ======================================================
# 1. KHỞI TẠO ENV VÀ GẮN LOGGING (FIX LOGGING TẠI ĐÂY)
# ======================================================
print("Đang khởi tạo môi trường và logger...")
env = InstacartEnv(train_sales, capacities, cfg.NUMBER_OF_PRODUCTS)

# [FIX]: Monitor phải bọc env ngay tại đây, TRƯỚC khi đưa vào model
# file log sẽ nằm ở /kaggle/working/logs/training_log_resume.monitor.csv
env = Monitor(env, filename=os.path.join(LOG_DIR, "training_log_resume"))

# ======================================================
# 2. TÌM CHECKPOINT ĐỂ RESUME
# ======================================================
if INPUT_CHECKPOINT_PATH is None:
    print("Đang tìm checkpoint trong Input...")
    candidates = []
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            if filename.endswith('.zip') and "ddpg" in filename:
                candidates.append(os.path.join(dirname, filename))
    if candidates:
        INPUT_CHECKPOINT_PATH = max(candidates, key=lambda x: int(''.join(filter(str.isdigit, x)) or 0))

# ======================================================
# 3. LOAD MODEL HOẶC TẠO MỚI
# ======================================================
# Cấu hình mạng bài báo (SGD/Tanh)
policy_kwargs = dict(
    activation_fn=nn.Tanh, 
    net_arch=dict(pi=[256, 256], qf=[256]),
    optimizer_class=th.optim.SGD,
    optimizer_kwargs=dict(momentum=0.8)
)
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

if INPUT_CHECKPOINT_PATH and os.path.exists(INPUT_CHECKPOINT_PATH):
    print(f"🔄 Đang resume từ: {INPUT_CHECKPOINT_PATH}")
    # Load model vào env đã có Monitor
    model = DDPG.load(INPUT_CHECKPOINT_PATH, env=env)
    
    # Cập nhật tham số an toàn
    model.learning_rate = CURRENT_LR
    model.max_grad_norm = 0.5 # Chống nan
    
    # Load Replay Buffer
    buf_path = INPUT_CHECKPOINT_PATH.replace(".zip", "_replay_buffer.pkl")
    if os.path.exists(buf_path):
        print("📥 Đang load Replay Buffer...")
        model.load_replay_buffer(buf_path)
    reset_timesteps = False
else:
    print("✨ Tạo model mới (Paper Architecture)...")
    model = DDPG(
        "MlpPolicy", env, action_noise=action_noise, verbose=1,
        learning_rate=0.005, # LR khởi điểm (chưa resume)
        batch_size=32, gamma=0.99,
        policy_kwargs=policy_kwargs,
        buffer_size=100000, 
        max_grad_norm=0.5, # Chống nan
        device='auto'
    )
    reset_timesteps = True

# ======================================================
# 4. START TRAINING
# ======================================================
steps_per_episode = env.unwrapped.max_steps
save_freq = 100 * steps_per_episode
callback = RollingCheckpointCallback(save_freq, CHECKPOINT_DIR, MODEL_PREFIX, max_keep=3)

print(f"🚀 Bắt đầu train thêm {STEPS_TO_RUN_THIS_SESSION} bước...")
model.learn(
    total_timesteps=STEPS_TO_RUN_THIS_SESSION, 
    callback=callback, 
    reset_num_timesteps=reset_timesteps
)
print("🏁 Hoàn tất phiên này!")

Đang khởi tạo môi trường và logger...
Đang tìm checkpoint trong Input...
🔄 Đang resume từ: /kaggle/input/ddgp-s-d-ngsgp/checkpoints/ddpg_instacart_paper_984500_steps.zip
Wrapping the env in a DummyVecEnv.
📥 Đang load Replay Buffer...


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


🚀 Bắt đầu train thêm 3000000 bước...
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 891      |
|    ep_rew_mean     | 698      |
| time/              |          |
|    episodes        | 1108     |
|    fps             | 160      |
|    time_elapsed    | 22       |
|    total_timesteps | 988064   |
| train/             |          |
|    actor_loss      | -71.2    |
|    critic_loss     | 0.845    |
|    learning_rate   | 0.001    |
|    n_updates       | 987962   |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 891      |
|    ep_rew_mean     | 695      |
| time/              |          |
|    episodes        | 1112     |
|    fps             | 163      |
|    time_elapsed    | 43       |
|    total_timesteps | 991628   |
| train/             |          |
|    actor_loss      | -71.6    |
|    critic_loss     | 0.124    |
|    learning_rate   | 0.001    |
|    n_upda